In [20]:
from pathlib import Path
import optuna
from optuna.samplers import CmaEsSampler
import subprocess
import pickle
import time

In [21]:
def objective(trial):
    global acopp_dir

    ants = trial.suggest_int("ants", 1, 500)
    alpha = trial.suggest_float("alpha", 0, 10)
    beta = trial.suggest_float("beta", 0, 10)
    rho = trial.suggest_float("rho", 0, 1)
    q0 = trial.suggest_float("q0", 0, 1)
    ptries = trial.suggest_int("ptries", 1, 5)

    cluster_size = trial.suggest_int("cluster_size", 10, 128)
    sector = trial.suggest_int("sector", 2, cluster_size)
    n_cluster = trial.suggest_int("n_cluster", 2, 1000 // cluster_size + 1)

    command = [
        'python3',
        f'{acopp_dir}/run.py',
        '--acopp_dir',
        str(acopp_dir),
        '--instance_name',
        'eil51_10_usw_10_03',
        '--aaco_nc',
        '--run_only',
        '--experiment',
        # '--no_log',
        '--sol_dir',
        '../../solutions/temp',
        '--silent',
        '1',
        '--random_seed',
        str(int(time.time())),
        '--alpha',
        str(alpha),
        '--beta',
        str(beta),
        '--rho',
        str(rho),
        '--q0',
        str(q0),
        '--ants',
        str(ants),
        '--ptries',
        str(ptries),
        '--sector',
        str(sector),
        '--cluster_size',
        str(cluster_size),
        '--n_cluster',
        str(n_cluster),
        # "--time",
        # "10"
    ]
    
    result = subprocess.run(command, capture_output=True)
    assert (
        result.returncode == 0
    ), f"""
command:
{' '.join(command)}
returncode: {result.returncode}
stderr:
{result.stderr.decode()}
stdout:
{result.stdout.decode()}
"""
    stdout_log = result.stdout.decode()
    profit = int(stdout_log)
    return profit

In [22]:
acopp_dir = Path("./")
build_command = [
    'python3',
    f'{acopp_dir}/run.py',
    '--acopp_dir',
    str(acopp_dir),
    '--build_only',
    '--experiment'
    ]
n_trials = 5000
n_jobs = 6
# n_trials = 2
# n_jobs = 2
save_path = Path("./study.pkl")

In [23]:
result = subprocess.run(build_command, capture_output=True)
assert (
    result.returncode == 0
), f"""
command:
{' '.join(build_command)}
returncode: {result.returncode}
stderr:
{result.stderr.decode()}
stdout:
{result.stdout.decode()}
"""
study = optuna.create_study(direction='maximize', sampler=CmaEsSampler)
study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs)

[I 2023-06-01 15:56:39,500] A new study created in memory with name: no-name-776ded0e-7ef1-483f-ad79-94aaafb9f357
[I 2023-06-01 15:56:50,344] Trial 0 finished with value: 210436.0 and parameters: {'ants': 110, 'alpha': 8.30568362612825, 'beta': 0.04884556746251212, 'rho': 0.9992350791706474, 'q0': 0.8784158129042516, 'ptries': 2, 'cluster_size': 82, 'sector': 5, 'n_cluster': 3}. Best is trial 0 with value: 210436.0.
[I 2023-06-01 15:56:50,646] Trial 1 finished with value: 216864.0 and parameters: {'ants': 172, 'alpha': 5.350979841350897, 'beta': 3.608813010590548, 'rho': 0.5547783935690562, 'q0': 0.48848294185134733, 'ptries': 3, 'cluster_size': 94, 'sector': 52, 'n_cluster': 2}. Best is trial 1 with value: 216864.0.


In [24]:
with open(save_path, "wb") as f:
    pickle.dump(study, f)